In [1]:
# - IMPORTS ---
import os
import numpy as np
import pandas as pd
from IPython.display import Markdown, display

In [2]:
# - FUNCTIONS ---
def compute_summary_stats_for_csv_file(
    csv_f:str,
    coeff_type: str,
    filter_columns: list
) -> pd.DataFrame:
    """
    Compute summary statistics for the given CSV file
    """
    df_table = pd.read_csv(csv_f)
    columns_oI = df_table.columns[2:-2]  # Select Only columns with numerical values
    
    if filter_columns != []:
        columns_oI = filter_columns

    # Filter the table
    df_table_corr = df_table[df_table["Coeff"] == f"{coeff_type}_corr_coeffs"]
    df_table_p = df_table[df_table["Coeff"] == f"{coeff_type}_p_values"]
    
    df_table_corr_f = pd.DataFrame(df_table_corr,
                                   columns = columns_oI) 

    # Check which coefficient have a p value lover than threshold (statistically significant)
    # TODO, use as filter: mask = (df_table_p_f < p_threhsold)  
    df_table_p_f = pd.DataFrame(df_table_p,
                                columns = columns_oI) 
    valid_df_table_corr = df_table_corr_f

    summary_statistics = {'mean': valid_df_table_corr.mean(),
                          'std': valid_df_table_corr.std(),
                          'min': valid_df_table_corr.min(),
                          '25% quantile': valid_df_table_corr.quantile(0.25),
                          '50% quantile': valid_df_table_corr.quantile(0.50),
                          '75% quantile': valid_df_table_corr.quantile(0.75),
                          'max': valid_df_table_corr.max(),
                          }

    return pd.DataFrame(summary_statistics).T

def filter_csv_files(sim_or_not, op_type, coeff_type, all_csv_files):
    check = lambda file_name, options: any(o in file_name for o in options)
    if sim_or_not != []:
        all_csv_files = [f for f in all_csv_files if check(f, sim_or_not)]
    if op_type != []:
        all_csv_files = [f for f in all_csv_files if check(f, op_type)]
    if coeff_type != []:
        all_csv_files = [f for f in all_csv_files if check(f, coeff_type)]
    return all_csv_files



In [3]:
# - PARAMETERS ---

# Import from parent folder
import sys
from pathlib import Path

# Add the parent directory to sys.path
print(Path.cwd())
#print(str(Path.cwd()).replace("notebooks", "configuration"))
#print("/home/user_id/WORK_Station/VOS/UVOS/iVOTS_Benchmark/configuration")
#sys.path.append(str(Path.cwd()).replace("notebooks", "configuration"))

from configuration.configuration_manager import ConfigManager

dataset_name = "lvos-val"  #"d17-val", "lvos-val"
method_name = "Ensemble_v3"#"UXMem_s012_base" #"Ensemble_v3" #'UXMem_s012_base' #'XMem_Ensemble_3_v2' #"XMem_Ensemble_2" #'QDMN'

dataset_name = "d17-val"  #"d17-val", "lvos-val"
method_name = "QDMN_S" #"UXMem_s012_base" #"Ensemble_v3" #'UXMem_s012_base' #'XMem_Ensemble_3_v2' #"XMem_Ensemble_2" #'QDMN'

#"UXMem_s012_base" #"XMem"
sim_or_not = ['noxsim_']  # 'yexsim_', 'noxsim_'
op_type = ['_sum_']  # '_mean_', '_median_', '_sum_'
coeff_type = "spearman"  # 'spearman', 'pearson'
temperature = 1.0
mask_H = 5

def gen_location_for_the_csv_file(_save_analysis_loc, mask_H=None):
    _save_analysis_loc = os.path.join(_save_analysis_loc, "raw_stats_HUB")
    if mask_H is not None:
        _save_analysis_loc = os.path.join(_save_analysis_loc, 
                                          f"mask_H_{mask_H}")
    else:
        _save_analysis_loc = os.path.join(_save_analysis_loc, "no_mask_H")

    return _save_analysis_loc


# Configuration/ Available methods and datasets
config = ConfigManager()
avail_datasets = config.get_all_available_datasets()
avail_methods = config.get_all_available_methods()
config['dataset_name'] = dataset_name
config['method_name'] = method_name
config_generator = config.get_my_configuration()

# TODO: adapt this !!
image_directory, gt_mask_directory = next(config_generator)
pred_mask_directory, logits_directory, softmax_directory = \
    next(config_generator)
entropy_directory, confusion_directory = next(config_generator)
preprocessing_directory = next(config_generator)
save_analysis_loc = config.get_iou_entropy_analysis_path()
loc_to_save_csv_results = gen_location_for_the_csv_file(save_analysis_loc, mask_H)

if temperature != 1.0:
    entropy_directory = entropy_directory + f'_Temp_{temperature}'
    loc_to_save_csv_results += f'_Temp_{temperature}'


raw_stats_path = loc_to_save_csv_results
root, end = raw_stats_path.split('/raw_stats_HUB/')
final_stats_path = os.path.join(root, "summary_stats", end)
stats_location = os.path.join(final_stats_path)
directories = os.listdir(stats_location)

if method_name == "QDMN":
    focus_on_columns = ["QAM_score_fdx", "QAM_score_fdx_obx"]

else:
    focus_on_columns = ["Total_H_base", "TR_H_base", "FR_H_base",
                        # "Total_H_masked_gt", "TR_H_masked_gt", "FR_H_masked_gt",
                        "Total_H_masked_pd", "TR_H_masked_pd", "FR_H_masked_pd"]

# - MAIN ---
summary_markdown = "\n---\n" 
all_csv_files = filter_csv_files(sim_or_not, op_type, coeff_type, directories)

type_table = "Markdown"

for csv_f in all_csv_files:
    csv_f = os.path.join(stats_location, csv_f)
    summary_table = compute_summary_stats_for_csv_file(csv_f, coeff_type, focus_on_columns)
    if "Markdown" == type_table:
        markdown_str = summary_table.to_markdown()
    elif "Latex" == type_table:
        markdown_str = summary_table.to_latex()
    else:
        markdown_str = summary_table.to_markdown()
    summary_markdown += f"\n### {csv_f} \n{markdown_str} \n"

summary_markdown = summary_markdown + "\n---"
display(Markdown(summary_markdown))

print("\n\n Results adapted to be pasted in a markdown file")
print(summary_markdown)

/home/user_id/WORK_Station/VOS/UVOS/iVOTS_Benchmark



---

### ../intermediate/iVOTS_benchmark/GT_mask/iou_entropy_analysis_HUB/QDMN_S/d17-val/summary_stats/mask_H_5/noxsim_sum_base.csv 
|              |   Total_H_base |   TR_H_base |   FR_H_base |   Total_H_masked_pd |   TR_H_masked_pd |   FR_H_masked_pd |
|:-------------|---------------:|------------:|------------:|--------------------:|-----------------:|-----------------:|
| mean         |     -0.0648034 |   0.0149762 |  -0.302456  |          0.00182079 |        0.108711  |        -0.235366 |
| std          |      0.543928  |   0.516579  |   0.512234  |          0.52237    |        0.498856  |         0.523951 |
| min          |     -0.800397  |  -0.754247  |  -0.96898   |         -0.815947   |       -0.678367  |        -0.96898  |
| 25% quantile |     -0.52414   |  -0.394658  |  -0.714729  |         -0.400606   |       -0.271447  |        -0.689714 |
| 50% quantile |     -0.217955  |  -0.126887  |  -0.412478  |         -0.0890466  |        0.0548714 |        -0.345646 |
| 75% quantile |      0.529172  |   0.5718    |   0.0690913 |          0.412574   |        0.543793  |         0.161797 |
| max          |      0.902277  |   0.920914  |   0.785589  |          0.928213   |        0.946422  |         0.882376 | 

### ../intermediate/iVOTS_benchmark/GT_mask/iou_entropy_analysis_HUB/QDMN_S/d17-val/summary_stats/mask_H_5/noxsim_sum_pd.csv 
|              |   Total_H_base |   TR_H_base |   FR_H_base |   Total_H_masked_pd |   TR_H_masked_pd |   FR_H_masked_pd |
|:-------------|---------------:|------------:|------------:|--------------------:|-----------------:|-----------------:|
| mean         |      -0.547974 |   -0.462147 |   -0.865005 |           -0.580908 |        -0.435488 |        -0.811721 |
| std          |       0.430551 |    0.451044 |    0.199808 |            0.407692 |         0.458927 |         0.276471 |
| min          |      -0.95587  |   -0.926721 |   -0.990556 |           -0.929555 |        -0.891257 |        -0.982957 |
| 25% quantile |      -0.836908 |   -0.752386 |   -0.958255 |           -0.857635 |        -0.753252 |        -0.954093 |
| 50% quantile |      -0.661139 |   -0.593231 |   -0.937371 |           -0.7207   |        -0.609236 |        -0.927533 |
| 75% quantile |      -0.523478 |   -0.375842 |   -0.840625 |           -0.544418 |        -0.292426 |        -0.807126 |
| max          |       0.604453 |    0.725686 |    0.324576 |            0.619985 |         0.805044 |         0.477142 | 

---



 Results adapted to be pasted in a markdown file

---

### ../intermediate/iVOTS_benchmark/GT_mask/iou_entropy_analysis_HUB/QDMN_S/d17-val/summary_stats/mask_H_5/noxsim_sum_base.csv 
|              |   Total_H_base |   TR_H_base |   FR_H_base |   Total_H_masked_pd |   TR_H_masked_pd |   FR_H_masked_pd |
|:-------------|---------------:|------------:|------------:|--------------------:|-----------------:|-----------------:|
| mean         |     -0.0648034 |   0.0149762 |  -0.302456  |          0.00182079 |        0.108711  |        -0.235366 |
| std          |      0.543928  |   0.516579  |   0.512234  |          0.52237    |        0.498856  |         0.523951 |
| min          |     -0.800397  |  -0.754247  |  -0.96898   |         -0.815947   |       -0.678367  |        -0.96898  |
| 25% quantile |     -0.52414   |  -0.394658  |  -0.714729  |         -0.400606   |       -0.271447  |        -0.689714 |
| 50% quantile |     -0.217955  |  -0.126887  |  -0.412478  |         -0.0890466  |